In [7]:
import numpy as np
import pandas as pd
import cvxpy as cp

## Set-Up:

- Dataframe Loading
- Base Tradeoff Matrix
- Team - Vector Conversion Functions
- Example Teams

In [8]:
payoff_df = pd.read_csv("MarvelRivals_Payoff_Matrix.csv")
hero_list = hero_list = list(payoff_df.iloc[:, 0])

payoff_df = pd.read_csv("MarvelRivals_Payoff_Matrix.csv",  index_col=0)
payoff_matrix = payoff_df.values

def team_to_vector(selected_heroes, hero_list):
    vector = np.zeros(len(hero_list), dtype=int)
    for i, hero in enumerate(hero_list):
        if hero in selected_heroes:
            vector[i] = 1
    return vector
    
def vector_to_team(binary_vector, hero_list):
    selected_heroes = [hero_list[i] for i in range(len(hero_list)) if binary_vector[i] == 1]
    return selected_heroes

Avengers = ["Black Widow", "Iron Man", "Thor", "Hulk", "Hawkeye", "Captain America"]
FantasticFour = ["Invisible Woman", "The Thing", "Human Torch", "Mister Fantastic", "Spider Man", "Wolverine"]
Guardians = ["Mantis", "Rocket Raccoon", "Adam Warlock", "Star Lord", "Groot", "Venom"]
Xmen = ["Magneto", "Psylocke", "Wolverine", "Magik", "Storm", "Namor"]
BestTeam = ["Magneto", "Doctor Strange", "The Punisher", "Scarlet Witch", "Cloak & Dagger", "Loki"]

In [9]:
rowPlayer = team_to_vector(BestTeam, hero_list)
columnPlayer = team_to_vector(Xmen, hero_list)

result = rowPlayer.T @ payoff_matrix @ columnPlayer
result

-11.69

## Base Question Implementation

It turns out, given an opponent team, it is relatively easy to figure out the optimal conter team using cvxpy.

In [10]:
def base_optimal_conter(opponent_team):
    x = cp.Variable(37, boolean=True)
    y = team_to_vector(opponent_team, hero_list)
    constraint_0 = [cp.sum(x) == 6]
    constraint_1 = [cp.sum(y) == 6]
    
    obj = cp.Maximize(x.T @ payoff_matrix @ y)
    prob = cp.Problem(obj,constraint_0+constraint_1)
    prob.solve()

    optimal_team = vector_to_team(x.value.round(), hero_list)
    expected_value = x.value.T @ payoff_matrix @ y

    print("Optimal Team:")
    for hero in optimal_team:
        print(f"-{hero}")
    print(f"Expected Utility Score (x^T * A * y) is: {expected_value:.2f}")
    return x.value

In [11]:
print(f"Best Conter to Fantastic Four is:")
FantasticFourConter = base_optimal_conter(FantasticFour)

Best Conter to Fantastic Four is:
Optimal Team:
-Captain America
-Peni Parker
-Thor
-Magik
-Storm
-Rocket Raccoon
Expected Utility Score (x^T * A * y) is: 21.91
